In [ ]:
print("Utviklingsmiljøet er klart!")

In [ ]:
import requests
import pandas as pd

# NASA POWER API-endepunkt
url = "https://power.larc.nasa.gov/api/temporal/daily/point"

# Gyldige API-parametere – Henter solinnstråling, bakketemperatur, luftfuktighet og lufttrykk
params = {
    "parameters": "ALLSKY_SFC_SW_DWN,TS,QV2M,PS",  
    "community": "RE",
    "longitude": 10.0,  # Endre til ønsket koordinat
    "latitude": 59.0,   # Endre til ønsket koordinat
    "start": "20230101",  # Startdato (YYYYMMDD)
    "end": "20231231",  # Sluttdato (YYYYMMDD)
    "format": "JSON"
}

# Hente data
response = requests.get(url, params=params)

# Sjekke respons
if response.status_code == 200:
    data = response.json()

    # Sjekke at dataene finnes i responsen
    if "properties" in data and "parameter" in data["properties"]:
        parameters = data["properties"]["parameter"]

        # Opprette DataFrame med utvalgte variabler
        df = pd.DataFrame.from_dict(parameters, orient="index").T
        df.index = pd.to_datetime(df.index)  # Konverterer indeksen til datoformat

        # Lagre til CSV
        df.to_csv("data/raw/nasa_extended_data.csv")
        print("Data lagret som nasa_extended_data.csv")

else:
    print(f"Feil: {response.status_code}, {response.text}")


In [ ]:
import requests
import pandas as pd

# API-endepunkt for Yr (sanntidsvær)
url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"

# Lokasjonsparametere (endre til ønsket sted)
params = {
    "lat": 59.0,  # Breddegrad
    "lon": 10.0   # Lengdegrad
}

# Headers - Krever User-Agent (Yr blokkerer anonyme forespørsler)
headers = {
    "User-Agent": "MiljodataApplikasjon/1.0 (kontakt@dinemail.com)"  # Endre til din kontaktinfo
}

# Hente data
response = requests.get(url, params=params, headers=headers)

# Sjekke respons
if response.status_code == 200:
    data = response.json()

    # Hente ut time-for-time værdata
    timeseries = data["properties"]["timeseries"]

    # Konvertere til DataFrame
    weather_data = []
    for entry in timeseries:
        time = entry["time"]
        details = entry["data"]["instant"]["details"]

        # Hent spesifikke værdata
        temp = details.get("air_temperature", None)
        wind_speed = details.get("wind_speed", None)
        wind_direction = details.get("wind_from_direction", None)
        precipitation = entry["data"].get("next_1_hours", {}).get("details", {}).get("precipitation_amount", 0)
        humidity = details.get("relative_humidity", None)
        pressure = details.get("air_pressure_at_sea_level", None)

        weather_data.append([time, temp, wind_speed, wind_direction, precipitation, humidity, pressure])

    # Opprette DataFrame med flere værparametere
    df = pd.DataFrame(weather_data, columns=["Time", "Temperature", "Wind Speed", "Wind Direction", "Precipitation", "Humidity", "Pressure"])

    # Konverter 'Time' til datetime-format
    df["Time"] = pd.to_datetime(df["Time"])

    # Lagre til CSV
    df.to_csv("yr_extended_weather_data.csv", index=False)
    print("Data lagret som yr_extended_weather_data.csv")

else:
    print(f"Feil: {response.status_code}, {response.text}")
